# Import the data

In [ ]:
import gdown

gdown.download(r'https://drive.google.com/file/d/1-7lEmQDmf06beJUD3NaQk82125NGqYpm/view?usp=drive_link', r'../../data/aerial_data/val_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw/view?usp=drive_link', r'../../data/aerial_data/val_images.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1vKs-th9eHU_oMXTasVm-FD9Dximer82Q/view?usp=drive_link', r'../../data/aerial_data/train_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n/view?usp=drive_link', r'../../data/aerial_data/train_images.npy',fuzzy=True)

# Loading numpy files from memory

In [ ]:
import os
import numpy as np
#system specific
folder = r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset"
train_images = np.load(os.path.join(folder, 'train_images.npy'))
train_labels = np.load(os.path.join(folder, 'train_labels.npy'))
val_images = np.load(os.path.join(folder, 'val_images.npy'))
val_labels = np.load(os.path.join(folder, 'val_labels.npy'))

In [ ]:
#inspecting data
print("train data:")
print("images shape:")
print(train_images.shape)
print("labels shape:")
print(train_labels.shape)

#print("test data:")
#print("images shape:")
#print(test_images.shape)
#print("labels shape:")
#print(test_labels.shape)

print("val data:")
print("images shape:")
print(val_images.shape)
print("labels shape:")
print(val_labels.shape)

print("image format check:")
print(train_images[0,:,:,0])

print("label check:")
print(train_labels[0])


# Performing Feature Extraction on Inception-V3, which is currently trained on image net
Adapted from: <https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/finetuning_torchvision_models_tutorial.ipynb#scrollTo=NiFT4EYmprqG>

# Training Parameters

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import  transforms

# Number of tree species categories
num_classes = 19

# Batch size for training (should be a power of 2 if training on the GPU and is limited by memory)
batch_size = 32

# Number of epochs to train for
num_epochs = 100

#True if we are just training the output layer of the network
feature_extract = True

print("cuda is available = ",torch.cuda.is_available())

# Defining function to reshape the model to account for the number of classes
By default Inception-v3 has 1000 classes as it is trained on imageNet, whilst our dataset has 19

# Initializing Custom Inception-V3Model


In [ ]:
from training_funcs import initialize_inception_model

model_ft, input_size = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)
print(model_ft)

# Formatting Loaded Data and creating data loader

In [ ]:
import numpy as np
#from PIL import Image
#converting numpy arrays to PIL images
#train_images_pil = [Image.fromarray(image) for image in train_images]
#val_images_pil = [Image.fromarray(image) for image in val_images]
#creating one hot encoding labels from numpy arrays
train_categories, train_labels_numeric = np.unique(train_labels, return_inverse=True)
val_categories, val_labels_numeric = np.unique(val_labels, return_inverse=True)

print("Train Categories:", train_categories)
print("Train Labels Numeric:", train_labels_numeric[0:10])
print("Val Categories:", val_categories)
print("Val Labels Numeric:", val_labels_numeric[0:10])
print(train_labels_numeric.dtype)


In [ ]:
if np.all(train_categories == val_categories):
    print("categories are consistent between train and val sets")
    categories = train_categories = val_categories
else:
    raise ValueError("Train and val categories don't match")

In [ ]:
# normalization for training and validation
# right now validation and training transform is the same and no augmentation
# Calculate mean and std for 4-channel images

#mean = np.mean(train_images[:,:,:,3], axis=(0, 1, 2)) / 255  # divide by 255 to normalize between 0 and 1
#std = np.std(train_images[:,:,:,3], axis=(0, 1, 2)) / 255
#print("Mean: ", mean)
#print("Std: ", std)

#Consider using standard ImageNet normalization values for RGB channels
#rgb_mean = [0.485, 0.456, 0.406] 
#rgb_std = [0.229, 0.224, 0.225]


In [ ]:
import training_funcs

#creating training and validation pytorch datasets
training_dataset = training_funcs.CustomDataset(train_images, train_labels_numeric, input_size)
val_dataset = training_funcs.CustomDataset(val_images, val_labels_numeric, input_size)

# Create training and validation dataloaders
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True),
                    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)}

# Training the model

In [ ]:
## Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("sending model to",device)
# Send the model to GPU
model_ft = model_ft.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

#Creating the Optimizer
if feature_extract:
  params_to_update = []
  for name,param in model_ft.named_parameters():
    if param.requires_grad==True:
      params_to_update.append(param)
      print("\t",name)
else:
  params_to_update = model_ft.parameters()

print("number of parameters to train =",len(params_to_update))
for i, param in enumerate(params_to_update):
  print("parameter {}:".format(i),param.shape)

optimizer_ft = optim.AdamW(params_to_update, lr=0.01, betas=(0.8, 0.99), weight_decay=0.001)
            

In [ ]:
from training_funcs import train_model
#fixing random seed for consistent results
torch.manual_seed(0)

#creating a tensorboard writer for logging
#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter("runs/train_run_test")
# Train and evaluate
model_ft, convergence_dict = train_model(model_ft,
                             dataloaders_dict,
                             criterion,
                             optimizer_ft,
                             num_epochs=num_epochs,
                             is_inception=True,
                             tensorboard_writer=None,
                             early_stopping=False,
                             device=device)


filename = "temp.pth"
if not os.path.exists(filename) or filename == "temp.pth":
    torch.save(model_ft.state_dict(),filename)
else:
    print("model file already exists")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
print(acc_hist)
print(loss_hist)

for entry in acc_hist:
  entry = entry.cpu().detach().numpy()

acc_hist_array = np.array(loss_hist)

plt.plot(acc_hist_array,"x-")
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("AdamW w. lr=0.01, betas=[0.8,0.99], w_decay=0.001")
plt.show()

# Random Search Implementation

In [ ]:
import csv
import torch
import numpy as np
from torch import optim
import random

def adamRandomSearch(model_ft, result_file, max_epochs=100, stop_tol=1e-3, patience=2, parameters=None):
    #initializing model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("sending model to",device)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()
    params_to_update = []
    
    for name,param in model_ft.named_parameters():
        if param.requires_grad==True:
            params_to_update.append(param)
            print("\t",name)
    
    # fixing the random seed for PyTorch
    torch.manual_seed(42)
    # fixing the random seed for NumPy
    np.random.seed(42)
    #fixing the random seed for the random module
    random.seed(42)
    try:
        count = 0
        while True:
            print("Random search iteration: ", count)
            # not fixing the random seed for choosing hyperparameters so that seperate runs are independent
            rng = np.random.default_rng()
            lr = rng.choice(parameters["lr"])
            beta1 = rng.choice(parameters["beta1"])
            beta2 = rng.choice(parameters["beta2"])
            weight_decay = rng.choice(parameters["weight_decay"])
            batch_size = int(rng.choice(parameters["batch_size"]))
            print("parameters: lr = ", lr, "beta1 = ", beta1, "beta2 = ", beta2, "weight_decay = ", weight_decay, "batch_size = ", batch_size)
            #setting optimizer and dataloaders
            optimizer = optim.AdamW(params_to_update, lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
            dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True),
                                'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)}
            # Train and evaluate
            model_ft, convergence_dict, epoch = train_model(model_ft,
                                                                dataloaders_dict,
                                                                criterion,
                                                                optimizer,
                                                                num_epochs=max_epochs,
                                                                is_inception=True,
                                                                early_stopping=True,
                                                                patience=patience,
                                                                stop_tol=stop_tol)
                
            # Append the result to a new line of the CSV file
            with open(result_file, "a", newline='') as file:
                writer = csv.writer(file)
                writer.writerow([epoch, max(convergence_dict["val_acc"]), min(convergence_dict["val_loss"]), lr, beta1, beta2, weight_decay, batch_size])                

            count += 1
            
    except KeyboardInterrupt:
        print("Random search stopped")
        return



### Running the Random Search

In [ ]:
#setting hyperparameters to search over
parameters={"lr":np.array([0.0001,0.0005,0.001,0.005,0.01,]),
            "beta1":np.array([0.7,0.75,0.8,0.85,0.9,0.95,0.99]),
            "beta2":np.array([0.9,0.95,0.99,0.995,0.999]),
            "weight_decay":np.array([0,0.0001,0.001,0.01,0.015,0.1,0.2]),
            "batch_size":np.array([8,16,32,64],dtype=int)}
#running random search. Can be stopped and started at whim, but the file "random_search.csv"
#must exist in the current working directory
adamRandomSearch(model_ft,"random_search_2.csv",max_epochs=50,stop_tol=1e-3,patience=3,parameters=parameters)

# Visualizing the dataset and model layers


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np

#visuallizing training data
# Select 20 random indices
random_indices = random.sample(range(train_images.shape[0]), 20)
plt.tight_layout()

print("----unnormalized images from training data----")
# Plot the images
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in zip(random_indices, axes.flatten()):
    ax.imshow(train_images[i,:,:,1:])
    ax.set_title(train_labels[i])  # Add title to each image
    ax.axis('off')

plt.show()
print("----normalized images from training data----")
# Plot the images
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in zip(random_indices, axes.flatten()):
    
    ax.imshow(training_dataset[i][0].permute(1, 2, 0))
    ax.set_title(train_labels[i])  # Add title to each image
    ax.axis('off')

plt.show()
print("----denormalized images from training data----")
unorm = transforms.Normalize(-np.divide(mean,std), 1/std)

fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in zip(random_indices, axes.flatten()):
    image = unorm(training_dataset[i][0])
    
    ax.imshow(image[1:,:,:].permute(1, 2, 0))
    ax.set_title(train_labels[i])  # Add title to each image
    ax.axis('off')

plt.show()

print("----first layer embeddings from training data----")
# Plot the images
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in zip(random_indices, axes.flatten()):
    image_4c = training_dataset[i][0]
    image_3c = model_ft.pre_layer.input_conv(image_4c)
    ax.imshow(image_3c.detach().permute(1, 2, 0))
    ax.set_title(train_labels[i])  # Add title to each image
    ax.axis('off')

plt.show()

print("----first layer embeddings from training data: unnormalized----")
# Plot the images
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in zip(random_indices, axes.flatten()):
    unorm2 = transforms.Normalize(-np.divide(mean[1:],std[1:]), 1/std[1:])
    image_4c = training_dataset[i][0]
    image_3c = model_ft.pre_layer.input_conv(image_4c)
    img = unorm2(image_3c.detach())
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(train_labels[i])  # Add title to each image
    ax.axis('off')

# print("train_images shape:",train_images.shape)
# for i in range(10):
#     writer.add_images('unnormalized_images, batch {}'.format(i),
#                         train_images[(i*20):(i*20+20),:,:,1:],
#                         i,
#                         dataformats='NHWC')

# for i in range(10):
#     writer.add_images('first_layer_embedding_normalized_3',
#                         training_dataset[i][0][:3,:,:], 
#                         i,
#                         dataformats='CHW'
#     )

# for i in range(10):
#     image_4c = training_dataset[i][0]
#     image_3c = model_ft.pre_layer.input_conv(image_4c)
#     writer.add_images('first_layer_embedding',
#                         image_3c, 
#                         i,
#                         dataformats='CHW'
#     )




In [ ]:
%reload_ext tensorboard

# Testing Trained Model

### Downloading test data

In [ ]:
import gdown

gdown.download(r'https://drive.google.com/file/d/1-4ZKGtGrWCEfaRDBobEgeQJmtdJIFobi/view?usp=drive_link', r'../../data/aerial_data/test_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1-01C3LjzGSz7QGzvharc52yZRwEgMzni/view?usp=drive_link', r'../../data/aerial_data/test_images.npy',fuzzy=True)

### Loading Test Data

In [ ]:
import numpy as np
from PIL import Image
import os
print(os.getcwd())
test_labels = np.load('../../data/aerial_data/test_labels.npy')
test_images = np.load('../../data/aerial_data/test_images.npy')

#converting numpy arrays into RGB PIL images, ignoring the IR channel for now
test_images_pil = [Image.fromarray(image) for image in test_images]
#creating one hot encoding labels from numpy arrays
test_categories, test_categories_numeric = np.unique(test_labels, return_inverse=True)

print("Test Categories:", test_categories)
print("train Categories:", train_categories)

test_dataset = CustomDataset(test_images_pil, test_categories_numeric, image_transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

### evaluating on test data

In [ ]:
import torch
import torch.nn.functional as F

model_file = 'training_run_2_4_channels.pth'
trained_model, _ = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)
trained_model.load_state_dict(torch.load(model_file))
trained_model.to(device)
trained_model.eval()

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = trained_model(images)
        #applying softmax to model logits. This is done implicitley during the training but not during evaluation
        outputs = F.softmax(outputs, dim=1)

        _, predicted = torch.max(outputs, 1)

        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print("Number of test predictions:", total_predictions)
print(f"Accuracy on the test dataset: {accuracy:.2%}")


In [ ]:
print(trained_model)

# Extracting logits from trained model

In [ ]:
import numpy as np
import torch

eval_images = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\aerial_99_images.npy")
eval_labels = np.load(r"C:\Users\bench\OneDrive\Documents\EMAT Year 3\MDM3\Phase C\ratio_adjusted_aerial_dataset\aerial_99_labels.npy")
model_file = r"C:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\training\aerial_data_training\training_run_3_filtered_data.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

trained_model, _ = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)
trained_model.load_state_dict(torch.load(model_file))
trained_model.to(device)
trained_model.eval()


eval_categories, eval_categories_numeric = np.unique(eval_labels, return_inverse=True)

test_dataset = CustomDataset(eval_images, eval_categories_numeric, image_transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

eval_logits = np.zeros((len(eval_labels), num_classes))
with torch.no_grad():
    i=0
    for image, _ in test_dataloader:
        image = image.to(device)
        output = trained_model(image)
        eval_logits[i,:] = output.cpu().numpy()
        i+=1

print(eval_logits.shape)
print(eval_logits[0,:])        

In [ ]:
np.save("aerial_99_logits.npy", eval_logits)